In [33]:
!python -m pip install https://github.com/schmiph2/pysepm/archive/master.zip

  Using cached https://github.com/schmiph2/pysepm/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://github.com/jfsantos/SRMRpy/archive/master.zip
  Preparing metadata (setup.py) ... done
  Using cached https://github.com/ludlows/python-pesq/archive/master.zip
  Preparing metadata (setup.py) ... done
  Using cached https://github.com/detly/gammatone/archive/master.zip
  Preparing metadata (setup.py) ... done


In [34]:
import sys
sys.path.append('../../')

In [35]:
import pysepm

In [36]:
import torch
import torchaudio
from denoiser.denoiser import pretrained
from denoiser.denoiser.dsp import convert_audio


In [37]:
import os
from glob import glob

In [38]:
# from scipy.io import wavfile
# # import sys
# # sys.path.append("../") 



# Load Audio Files

In [39]:
DATA_DIR = "/home/ubuntu/efs/IntroDL/denoiser/dataset"
CLEAN_DIR = os.path.join(DATA_DIR, "starter_clean/clean")
NOISY_DIR = os.path.join(DATA_DIR, "starter_noisy/noisy")



In [40]:
clean_files = sorted(glob(CLEAN_DIR + "/*"))
noisy_files = sorted(glob(NOISY_DIR + "/*"))

`clean_files` and `noisy_files` does not have same order of files ; i.e. the i-th file of `clean_files` does not correspond to the i-th files of `noisy_files`. Hence, we should make the file id of the i-th file of `clean_files` and `noisy_files` correspond to each other. 

In [41]:
i = 100
name = os.path.basename(noisy_files[i]).split('_')[-1]
print(name)

142.wav


In [42]:
clean_dict = {os.path.basename(name).split('_')[-1]:name for name in clean_files}
noisy_dict = {os.path.basename(name).split('_')[-1]:name for name in noisy_files}


In [43]:
print(len(clean_files) == len(noisy_files))
print(set(clean_dict.keys()) == set(noisy_dict.keys()))
# Check that the set of file id of clean files and noisy files are exactly the same

True
True


In [44]:
clean_files = []
noisy_files = []
for fileid in clean_dict.keys() : 
    clean_files.append(clean_dict[fileid])
    noisy_files.append(noisy_dict[fileid])


In [45]:
print(noisy_files[:5])
print(clean_files[:5])

['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp126_3Wjw0nadnM4_snr15_tl-22_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp198_bus_56903_0_snr0_tl-32_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp535_washer_460549_1_snr2_tl-35_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp228_vacuum_379760_1_snr1_tl-19_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp591_clatter_noise_166453__zabuhailo__metalworking-hammer-5_5_snr9_tl-27_fileid_105.wav']
['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/s

In [46]:
print(noisy_files[-5:])
print(clean_files[-5:])

['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp644_barking_25887_0_snr5_tl-28_fileid_89.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp81_car_74675_2_snr18_tl-22_fileid_9.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp177_0kuIP7fLNpM_snr6_tl-26_fileid_90.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp383_babble_188218_15_snr6_tl-18_fileid_94.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp219_wind_344887_0_snr10_tl-28_fileid_96.wav']
['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_89.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_9.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_90.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_94.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_96.wav']


We have just checked that the order of files in the list are matched for `clean_files` and `noisy_files`.

In [47]:
clean_wav, clean_sr = torchaudio.load(clean_files[0])
noisy_wav, noisy_sr = torchaudio.load(noisy_files[0])

In [48]:
clean_wav

tensor([[ 0.0731,  0.0881,  0.1040,  ..., -0.0025, -0.0023, -0.0017]])

In [49]:
noisy_wav

tensor([[0.0877, 0.1019, 0.1178,  ..., 0.0010, 0.0010, 0.0011]])

In [50]:
print(clean_sr)
print(noisy_sr)

16000
16000


In [51]:
clean_wav.shape

torch.Size([1, 160000])

Note that in the [example notebook](https://github.com/schmiph2/pysepm/blob/master/examples/examplesForCalculatingMeasures.ipynb) provided by the github repository of **pysepm**, the type and the shape of the input are given as numpy array and $(N,)$ where $N$ is equal to the product of `sample_rate` and `duration of audio file (in seconds)`

In [52]:
clean_wav = clean_wav.numpy().reshape(-1)
noisy_wav = noisy_wav.numpy().reshape(-1)

In [53]:
print(type(clean_wav))
print(clean_wav.shape)

<class 'numpy.ndarray'>
(160000,)


Bring the file list of enhanced audio files in the same order as the clean and noisy files list.

In [54]:
RESULT_DIR = "/home/ubuntu/efs/IntroDL/denoiser/results"

demucs_enhanced_files = sorted(glob(RESULT_DIR + "/Demucs_denoised/*"))
fullsubnet_enhanced_files = sorted(glob(RESULT_DIR + "/FullSubNet_denoised/*"))
demucs_enhanced_dict = {os.path.basename(name).split('_')[-1]:name for name in demucs_enhanced_files}
fullsubnet_enhanced_dict = {os.path.basename(name).split('_')[-1]:name for name in fullsubnet_enhanced_files}



In [55]:
demucs_enhanced_files = []
fullsubnet_enhanced_files = []
for fileid in clean_dict.keys() : 
    demucs_enhanced_files.append(demucs_enhanced_dict[fileid])
    fullsubnet_enhanced_files.append(fullsubnet_enhanced_dict[fileid])

In [56]:
print(noisy_files[:5])
print(clean_files[:5])
print(demucs_enhanced_files[:5])
print(fullsubnet_enhanced_files[:5])

['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp126_3Wjw0nadnM4_snr15_tl-22_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp198_bus_56903_0_snr0_tl-32_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp535_washer_460549_1_snr2_tl-35_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp228_vacuum_379760_1_snr1_tl-19_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_noisy/noisy/clnsp591_clatter_noise_166453__zabuhailo__metalworking-hammer-5_5_snr9_tl-27_fileid_105.wav']
['/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_0.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_101.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_102.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/starter_clean/clean/clean_fileid_104.wav', '/home/ubuntu/efs/IntroDL/denoiser/dataset/s

In [57]:
demucs_enhanced_wav, demucs_enhanced_sr = torchaudio.load(demucs_enhanced_files[0])
fullsubnet_enhanced_wav, fullsubnet_enhanced_sr = torchaudio.load(fullsubnet_enhanced_files[0])

In [58]:
demucs_enhanced_wav

tensor([[ 0.0834,  0.0984,  0.1180,  ..., -0.0008, -0.0008, -0.0006]])

In [59]:
fullsubnet_enhanced_wav

tensor([[0.0718, 0.0818, 0.0923,  ..., 0.0002, 0.0005, 0.0007]])

In [60]:
print(demucs_enhanced_sr)
print(fullsubnet_enhanced_sr)

16000
16000


In [61]:
demucs_enhanced_wav = demucs_enhanced_wav.numpy().reshape(-1)
fullsubnet_enhanced_wav = fullsubnet_enhanced_wav.numpy().reshape(-1)


# Calc Measures

## STOI

In [62]:
pysepm.stoi(clean_wav, noisy_wav, clean_sr)

0.9807254867015377

In [63]:
pysepm.stoi(clean_wav, demucs_enhanced_wav, clean_sr)

0.9933100283139101

In [64]:
pysepm.stoi(clean_wav, fullsubnet_enhanced_wav, clean_sr)

0.9917838243037159

Conjecture by Taeyoung

STOI :  
- somehow measures the correlation between two waveforms
- the higher the better

## PESQ

In [65]:
pysepm.pesq(clean_wav, noisy_wav, clean_sr)[1]

2.3495934009552

In [66]:
pysepm.pesq(clean_wav, demucs_enhanced_wav, clean_sr)[1]

3.5990958213806152

In [67]:
pysepm.pesq(clean_wav, fullsubnet_enhanced_wav, clean_sr)[1]

3.5924763679504395

Conjecture by Taeyoung

PESQ :  
- Subtract disturbance from $a_0 = 4.5$
- Closer to 4.5 the better

## fwSNRseg

In [68]:
pysepm.fwSNRseg(clean_wav, noisy_wav, clean_sr)

20.69275673110846

In [69]:
pysepm.fwSNRseg(clean_wav, demucs_enhanced_wav, clean_sr)

22.317664763495486

In [70]:
pysepm.fwSNRseg(clean_wav, fullsubnet_enhanced_wav, clean_sr)

23.481140233098415

Conjecture by Taeyoung

Signal to noise ratio :  
- first argument is clean, which is signal 
- the second argument is noisy, which is signal + noise.
- Noise should be small, so high SNR is desired.

In [71]:
STOI_noisy = []
STOI_demucs_enhanced = []
STOI_fullsubnet_enhanced = []
PESQ_noisy = []
PESQ_demucs_enhanced = []
PESQ_fullsubnet_enhanced = []
FWSNR_noisy = []
FWSNR_demucs_enhanced = []
FWSNR_fullsubnet_enhanced = []

for i in range(len(clean_files)) :
    clean_wav, clean_sr = torchaudio.load(clean_files[i])
    noisy_wav, noisy_sr = torchaudio.load(noisy_files[i])
    demucs_enhanced_wav, demucs_enhanced_sr = torchaudio.load(demucs_enhanced_files[i])
    fullsubnet_enhanced_wav, fullsubnet_enhanced_sr = torchaudio.load(fullsubnet_enhanced_files[i])
    
    clean_wav = clean_wav.numpy().reshape(-1)
    noisy_wav = noisy_wav.numpy().reshape(-1)
    demucs_enhanced_wav = demucs_enhanced_wav.numpy().reshape(-1)
    fullsubnet_enhanced_wav = fullsubnet_enhanced_wav.numpy().reshape(-1)

    STOI_noisy.append(pysepm.stoi(clean_wav, noisy_wav, clean_sr))
    STOI_demucs_enhanced.append(pysepm.stoi(clean_wav, demucs_enhanced_wav, clean_sr))
    STOI_fullsubnet_enhanced.append(pysepm.stoi(clean_wav, fullsubnet_enhanced_wav, clean_sr))

    PESQ_noisy.append(pysepm.pesq(clean_wav, noisy_wav, clean_sr)[1])
    PESQ_demucs_enhanced.append(pysepm.pesq(clean_wav, demucs_enhanced_wav, clean_sr)[1])
    PESQ_fullsubnet_enhanced.append(pysepm.pesq(clean_wav, fullsubnet_enhanced_wav, clean_sr)[1])

    FWSNR_noisy.append(pysepm.fwSNRseg(clean_wav, noisy_wav, clean_sr))
    FWSNR_demucs_enhanced.append(pysepm.fwSNRseg(clean_wav, demucs_enhanced_wav, clean_sr))
    FWSNR_fullsubnet_enhanced.append(pysepm.fwSNRseg(clean_wav, fullsubnet_enhanced_wav, clean_sr))


In [81]:
import numpy as np

In [82]:
print("mean and stdev of STOI for noisy files are {:.4f} , {:.4f}".format(np.mean(STOI_noisy),np.std(STOI_noisy)))
print("mean and stdev of STOI for DeMucs enhanced files are {:.4f} , {:.4f}".format(np.mean(STOI_demucs_enhanced),np.std(STOI_demucs_enhanced)))
print("mean and stdev of STOI for FullSubNet enhanced files are {:.4f} , {:.4f}".format(np.mean(STOI_fullsubnet_enhanced),np.std(STOI_fullsubnet_enhanced)))
print("mean and stdev of PESQ for noisy files are {:.4f} , {:.4f}".format(np.mean(PESQ_noisy),np.std(PESQ_noisy)))
print("mean and stdev of PESQ for DeMucs enhanced files are {:.4f} , {:.4f}".format(np.mean(PESQ_demucs_enhanced),np.std(PESQ_demucs_enhanced)))
print("mean and stdev of PESQ for FullSubNet enhanced files are {:.4f} , {:.4f}".format(np.mean(PESQ_fullsubnet_enhanced),np.std(PESQ_fullsubnet_enhanced)))
print("mean and stdev of fwSNRseg for noisy files are {:.4f} , {:.4f}".format(np.mean(FWSNR_noisy),np.std(FWSNR_noisy)))
print("mean and stdev of fwSNRseg for DeMucs enhanced files are {:.4f} , {:.4f}".format(np.mean(FWSNR_demucs_enhanced),np.std(FWSNR_demucs_enhanced)))
print("mean and stdev of fwSNRseg for FullSubNet enhanced files are {:.4f} , {:.4f}".format(np.mean(FWSNR_fullsubnet_enhanced),np.std(FWSNR_fullsubnet_enhanced)))


mean and stdev of STOI for noisy files are 0.9152 , 0.0647
mean and stdev of STOI for DeMucs enhanced files are 0.9652 , 0.0328
mean and stdev of STOI for FullSubNet enhanced files are 0.9641 , 0.0350
mean and stdev of PESQ for noisy files are 1.5822 , 0.4576
mean and stdev of PESQ for DeMucs enhanced files are 2.6447 , 0.6332
mean and stdev of PESQ for FullSubNet enhanced files are 2.8885 , 0.6719
mean and stdev of fwSNRseg for noisy files are 12.6234 , 4.9981
mean and stdev of fwSNRseg for DeMucs enhanced files are 17.1348 , 3.7549
mean and stdev of fwSNRseg for FullSubNet enhanced files are 16.9617 , 4.3833


In [83]:
import numpy as np
import pandas as pd

In [84]:
table = pd.DataFrame(columns = ["metric", "stat", "Noisy", "Demuc", "FullSubNet"])

In [85]:
table["metric"] = ["STOI", "PESQ", "fwSNRSeg"] * 2
table['stat'] = ['mean'] * 3 + ['std'] * 3

In [86]:
table = table.sort_values(['metric'])
table = table.set_index(['metric', 'stat'])


In [87]:
table.iloc[0] = [np.mean(PESQ_noisy), np.mean(PESQ_demucs_enhanced), np.mean(PESQ_fullsubnet_enhanced)]
table.iloc[1] = [np.std(PESQ_noisy), np.std(PESQ_demucs_enhanced), np.std(PESQ_fullsubnet_enhanced)]
table.iloc[2] = [np.mean(STOI_noisy), np.mean(STOI_demucs_enhanced), np.mean(STOI_fullsubnet_enhanced)]
table.iloc[3] = [np.std(STOI_noisy), np.std(STOI_demucs_enhanced), np.std(STOI_fullsubnet_enhanced)]
table.iloc[4] = [np.mean(FWSNR_noisy), np.mean(FWSNR_demucs_enhanced), np.mean(FWSNR_fullsubnet_enhanced)]
table.iloc[5] = [np.std(FWSNR_noisy), np.std(FWSNR_demucs_enhanced), np.std(FWSNR_fullsubnet_enhanced)]

In [88]:
table

Noisy      Demuc FullSubNet
metric   stat                                
PESQ     mean  1.582239   2.644714    2.88852
         std   0.457569   0.633244   0.671924
STOI     mean  0.915185    0.96523   0.964117
         std   0.064698   0.032818    0.03496
fwSNRSeg mean  12.62337  17.134819  16.961693
         std   4.998137   3.754918   4.383323

In [89]:
pd.DataFrame.to_csv(table,"./EvalMetrics/table.csv" )